In [1]:
import pandas as pd

In [125]:
sequences = pd.read_csv('data/kit_sequenzen.csv', header=0)

In [126]:
annotation = pd.read_csv('data/kit_annotation.csv', header=0)

In [165]:
df = pd.DataFrame(columns=['ID', 'sequence', 'coordinates', 'start_coordinates'])

In [166]:
all_items = []

for row in range(0, len(sequences)):
    seq = []
    ID = sequences.at[row, 'ID']
    for item in sequences.at[row,'sequence'].split(','):
        if item not in seq:
            seq.append(item)
    df.at[row, 'ID'] = ID
    df.at[row, 'sequence'] = seq
    all_items.append(seq)

In [167]:
item_set = set([item for sublist in all_items for item in sublist])

In [168]:
for row in range(0, len (df)):
    locations_dict = {}
    ID = df.at[row, 'ID']
    filtered_df = annotation.loc[annotation['annotation'] == 'start'].set_index('ID')
    
    for item in item_set:
        locations_dict[item] = [filtered_df.at[ID, str(item) + '_x'],
                               filtered_df.at[ID, str(item) + '_y'], 2.0]
    df.at[row, 'coordinates'] = locations_dict

In [169]:
for row in range(0, len(df)):
    subject_locations = []
    ID = df.at[row, 'ID']
    filtered_annotation = annotation.loc[annotation['ID'] == ID].reset_index()
    
    for row2 in range(0, len(filtered_annotation)):
        if filtered_annotation.at[row2, 'annotation'] == 'start':
            subject_locations.append([filtered_annotation.at[row2, 'head_x'],
                                      filtered_annotation.at[row2, 'head_y'], 2])
        elif filtered_annotation.at[row2, 'annotation'] == 'nehmen_objekte':
            subject_locations.append([filtered_annotation.at[row2 - 1, 'head_x'],
                                     filtered_annotation.at[row2 - 1, 'head_y'], 2])
    
    #print(subject_locations)
    df.at[row, 'start_coordinates'] = subject_locations

In [184]:
# TODO: fix start coordinates if len seq > len start_coordinates
# (multiple items taken together?)